<a href="https://colab.research.google.com/github/hiya906/my-machine-learning/blob/master/Day_3_Tensorflow_2_0_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **사용법**

1.   우측 상단 '로그인'
2.   좌측 상단 '실습 모드에서 열기'


※ 각각의 셀은 셀 좌측 상단 실행 버튼을 통해 실행할 수 있습니다.

※ 실행 중 '경고: 이 노트는 Google에서 작성하지 않았습니다.'라는 창이 뜰 경우, '실행 전에 모든 런타임 재설정'란에 체크 후 '무시하고 계속하기'를 하시면 됩니다.

# Tensorflow 2.0 Example

Tensorflow 2.0 API => https://www.tensorflow.org/versions/r2.0/api_docs/python/tf?hl=ko 

## tensorflow 2.0 설치 및 import

In [1]:
!pip install tensorflow-gpu==2.0.0-beta1
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

     |████████████████████████████████| 348.9MB 55kB/s 
     |████████████████████████████████| 3.1MB 35.8MB/s 
     |████████████████████████████████| 501kB 46.0MB/s 


## tf.data.Dataset 객체 활용하기

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print(x_train.shape, x_test.shape)
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
print(x_train.shape, x_test.shape)

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) (10000, 28, 28)
(60000, 28, 28, 1) (10000, 28, 28, 1)


## tf.keras API 모델 만들기


 ![대체 텍스트](https://postfiles.pstatic.net/MjAxOTA3MjFfMTkz/MDAxNTYzNzEwNDI4ODUy.9yzokK56bxXvT482dBeu5XYvJvI0jiIyV34mtI4cOBAg.SYmNQn3WiVrHNQJBQdr3FEzbJaXcMbbV67ozUXJoTVIg.PNG.er2357/Sample.png?type=w773)




In [0]:
class MyModel(tf.keras.Model):
  # __init__() 함수는 객체가 생성되면 자동으로 실행되는 함수입니다.
  # 예를 들어, 아래와 같이 객체를 생성한 후
  # model = MyModel() <-- __init__()함수가 호출됨
  # __init__() 함수에선 모델을 구성하는 layer들을 정의합니다.
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
    self.flatten = tf.keras.layers.Flatten()
    self.d1 = tf.keras.layers.Dense(128, activation='relu')
    self.d2 = tf.keras.layers.Dense(10, activation='softmax')
    
  # call 함수는 객체 생성후 객체가 함수의 형태로 호출되면 자동으로 실행되는 함수입니다.
  # 예를 들어, 아래와 같이 객체를 생성한 후
  # model = MyModel()
  # model(x) <-- call()함수가 호출됨
  # call 함수에선 __init__() 함수에서 생성한 layer들의 순서를 조정합니다.
  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

## tf.keras API 손실함수와 옵티마이저 함수 선택하기

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


## tf.keras API 모델 훈련 방법 정의


In [0]:
@tf.function
def train_step(images, labels):
  # 아래 tf.GradientTape 안에서 실행된 모든 연산을 테이프에 "기록"합니다.
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  
  # 테이프 내에 기록된 연산에 대한 gradient를 계산합니다.
  gradients = tape.gradient(loss, model.trainable_variables)
  
  # 옵티마이저로 계산한 gradient를 모델 변수에 적용합니다.
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  
  # 앞서 정의한 손실함수로 손실을 계산합니다.
  train_loss(loss)
  
  # 모델의 정확도를 평가합니다.
  train_accuracy(labels, predictions)


## tf.keras API 모델 테스트 방법 정의

In [0]:
@tf.function
def test_step(images, labels):
  	predictions = model(images)
  	t_loss = loss_object(labels, predictions)

  	test_loss(t_loss)
  	test_accuracy(labels, predictions)


## tf.keras API 모델 학습 및 테스트

In [7]:
EPOCHS = 5
for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)
  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print(template.format(epoch+1,	train_loss.result(), train_accuracy.result()*100, test_loss.result(), test_accuracy.result()*100))


에포크: 1, 손실: 1.6518089771270752, 정확도: 81.02833557128906, 테스트 손실: 1.5877132415771484, 테스트 정확도: 87.33999633789062
에포크: 2, 손실: 1.5816655158996582, 정확도: 88.05583190917969, 테스트 손실: 1.537205457687378, 테스트 정확도: 92.45999908447266
에포크: 3, 손실: 1.5469955205917358, 정확도: 91.51277160644531, 테스트 손실: 1.5187485218048096, 테스트 정확도: 94.29666137695312
에포크: 4, 손실: 1.528463363647461, 정확도: 93.3558349609375, 테스트 손실: 1.5089844465255737, 테스트 정확도: 95.24749755859375
에포크: 5, 손실: 1.516891598701477, 정확도: 94.50466918945312, 테스트 손실: 1.5035581588745117, 테스트 정확도: 95.80199432373047
